# 1. GPT 모델

기존의 encoder-decoder 구조인 Transformer 모델을 변형하여 GPT 모델 구조를 만들기 위해서는 아래와 같은 변경이 필요합니다. 이때 빨간색 박스는 사라져야할 컴포넌트이며, 초록색 박스는 변경되어야 하는 컴포넌트를 의미합니다.
![](https://github.com/minkj1992/ai/blob/main/static/Untitled-2024-06-21-1948.png?raw=true)

openai에서 발표한 [Improving Language Understanding by Generative Pre-Training](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf) 논문에 따르면, decoder 모델을 토대로 generalized한 pre-trained 모델을 만듭니다. 그러므로 transformer의 사라져야할 부분은 아래와 같습니다.

#### 사라져야할 부분
> 빨간 네모
1. `Encoder`
2. `encoder-decoder attention layer`


위 논문에 따르면 변경되어야할 부분은 아래와 같습니다.

#### 변경되어야 할 부분
> 초록 네모

1. Positional encoding -> Positional embedding
2. Target input -> `Task-specific input transformations`

해당 논문에서는 또한 기존의 sin,cos을 활용한 positional 계산 방식을 positional embedding (max_sentence_len, d_model)을 생성하여 학습되도록 합니다. 

Target input 또한 달라져야 하는데, 이번 시간에는 pretrain + fine tunning 총 2번 학습했던 논문과 달리 1번만 학습할 예정이기 때문에, input을 변형해서 넣어주도록 하였습니다. 

현재 데이터 셋은 사용자의 질문인, df['Q']와 챗봇의 답변인 df['A']가 존재하며, 이 두 데이터 모두 챗봇에 필요하며 부족한 데이터를 고려하여, 합쳐서 학습해주도록 하겠습니다.


In [1]:
from tqdm.notebook import tqdm
tqdm.pandas()
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = os.getenv('HOME')+'/aiffel/transformer_chatbot/data/ChatbotData .csv' 
origin = pd.read_csv(path) 

print(origin.shape)
origin.sample(15)

(11823, 3)


,Q,A,label
7078,역시 못잊겠습니다.,영원한 아픔은 없을 거예요.,1
10048,서로 사랑하면 다 될 줄 알았어,"살아가는데 사랑이 큰 힘이 되지만, 모든걸 다 해결해주지는 않아요.",2
7724,이제는 잡을 수가 없네,마지막을 아는 것도 필요해요.,1
899,내일 수학여행가!,친구들과 좋은 추억 만들고 오세요.,0
6085,당신도 저만큼 아팠으면 좋겠습니다.,이런 말을 하면서 당신이 아프지 않았으면 좋겠어요.,1
1741,몸이 여러 개면 좋겠다,그러면 못할 게 없겠네요.,0
1757,무리에 잘못 낀 듯,지금도 늦지 않았어요.,0
4693,카드 다 부러뜨려 버려야겠다.,소비 조절을 못하면 없애는 것도 방법이 될 거예요.,0
2548,손님 오는 거 싫어,밖에서 만나보세요.,0
11768,학생일 때 썸 괜찮을까,지금은 지나면 돌아오지 않아요.,2


# 1. Preprocess

In [3]:
tmp = origin.copy()

df_q = pd.DataFrame({
    'text': tmp['Q']
})
df_a = pd.DataFrame({
    'text': tmp['A']
})

df = pd.concat([df_q, df_a], ignore_index=True)

print(df.shape)
df.sample(15)

(23646, 1)


,text
12479,진짜 하고 싶은 걸 찾아보세요.
2290,사업 구상하고 있어
13049,행복은 마음 가짐에 있어요.
9103,긴 머리가 잘 어울렸던 그녀
1436,레시피대로 한 거 같은데 왜 맛이 없지?
8000,전여친이 나보다 나은 사람이라는 생각이 자꾸 들어
14879,그냥 잊어버리세요.
15571,벌써 그러면 안돼요.
8002,전여친한테 남친이 생겼네
16484,당신은 생각보다 큰 사람이에요.


In [5]:
def preprocess(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

def postprocess(df):
    df.replace('', np.nan, inplace=True)
    df.dropna(subset=['text'], inplace=True)
    df.to_csv('./gpt.csv')

print(df.shape)
df.drop_duplicates(subset = ['text'], inplace = True)
df['text'] = df['text'].progress_apply(preprocess)
postprocess(df)
print(df.shape)

(19436, 1)


  0%|          | 0/19413 [00:00<?, ?it/s]

(19413, 1)


## 2. Data prepare

- 질문,답변 각자 적절한 maximum 크기만 남기고 지운다.
- 모두 하나의 col으로 합쳐서 train한다. (Q, A말투 모두 학습)


## (WIP) 추후 도입 방법
1. Genralized Pretrain
2. Fine tunning

두가지를 수행하기 위해서 2가지에 필요한 데이터를 준비하겠습니다.

1번을 위해서는 하나의 row에 속한 df['Q']와 df['A']를 2개의 row로 나눠서 df['text']로 만들어 0...i -> i+1예측 teacher forcing을 합니다.

이후 2번은 `[SOS] Q [DELIM] A_set [EOS]`형태의 문장을 A_set만큼 만들어, 정답인 A를 예측하도록 하는 softmax를 생성, 이를 위해서는 정답이 아닌 negative sampling이 필요해서, 다음 기회에 시도


In [6]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if len(s.split()) <= max_len:
            cnt = cnt + 1
    print(
        "전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s"
        % (max_len, (cnt / len(nested_list)))
    )

    
len_text = [len(s.split()) for s in df['text']]
max_len_text = 9
print("text의 최소 길이 : {}".format(np.min(len_text)))
print("text의 최대 길이 : {}".format(np.max(len_text)))
print("text의 평균 길이 : {}".format(np.mean(len_text)))
below_threshold_len(max_len_text, df["text"])

text의 최소 길이 : 1
text의 최대 길이 : 24
text의 평균 길이 : 4.318085818781229
전체 샘플 중 길이가 9 이하인 샘플의 비율: 0.9841858548395405


In [7]:
before = len(df)
def is_within(text, max_len):
    return len(text.split()) <= max_len
_filter = df["text"].apply(is_within, max_len=max_len_text)
df = df[_filter]
print("전체 샘플수 :", (len(df)))
print(f"삭제된 샘플수: {before - len(df)}")

전체 샘플수 : 19106
삭제된 샘플수: 307


In [8]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    df["text"],
    target_vocab_size=2**13,
)
SOS, EOS = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("SOS 번호 :", [tokenizer.vocab_size])
print("EOS 번호 :", [tokenizer.vocab_size + 1])
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

SOS 번호 : [8817]
EOS 번호 : [8818]
8819


In [9]:
def tokenize(texts):
    tokenized = []
    max_len = 0
    for text in texts:
        tokenized_txt = SOS + tokenizer.encode(text) + EOS
        max_len = max(max_len, len(tokenized_txt))
        tokenized.append(tokenized_txt)

    # max_length 으로 모든 데이터셋을 패딩
    return tf.keras.preprocessing.sequence.pad_sequences(
        tokenized, maxlen=max_len, padding="pre"
    ), max_len


texts, MAX_LENGTH = tokenize(df["text"])
print(MAX_LENGTH, len(texts))

20 19106


In [10]:
print(texts[0].shape, len(texts))

(20,) 19106


In [11]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': texts[:,:-1], # eos 토큰 제외
    },
    {
        'outputs': texts[:, 1:] # sos 토큰 제외
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [12]:
for data in dataset.take(1):
    print(data)

({'inputs': <tf.Tensor: shape=(64, 19), dtype=int32, numpy=
array([[   0,    0,    0, ..., 5396, 6776,  313],
       [   0,    0,    0, ..., 4927, 3307,   72],
       [   0,    0,    0, ...,   25, 8134, 1390],
       ...,
       [   0,    0,    0, ..., 6578,  211,  148],
       [   0,    0,    0, ...,  388,  185,    1],
       [   0,    0,    0, ..., 1395, 2760,  533]], dtype=int32)>}, {'outputs': <tf.Tensor: shape=(64, 19), dtype=int32, numpy=
array([[   0,    0,    0, ..., 6776,  313, 8818],
       [   0,    0,    0, ..., 3307,   72, 8818],
       [   0,    0,    0, ..., 8134, 1390, 8818],
       ...,
       [   0,    0,    0, ...,  211,  148, 8818],
       [   0,    0,    0, ...,  185,    1, 8818],
       [   0,    0,    0, ..., 2760,  533, 8818]], dtype=int32)>})


# 모델 학습

In [13]:
def scaled_dot_product_attention(query, key, value, mask):
    """
    query: (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    key: (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    value: (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    padding_mask : (batch_size, 1, 1, key의 문장 길이)
    """

    matmul_qk = tf.matmul(query, key, transpose_b=True)

    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    if mask is not None:
        logits += mask * -1e9  # FYI, 0은 softmax에서 양수값을 가진다.

    attention_weights = tf.nn.softmax(logits, axis=-1)

    # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    output = tf.matmul(attention_weights, value)
    return output, attention_weights

In [14]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        # d_model을 num_heads로 나눈 값.
        # 논문 기준 : 64 (512 // 8)
        self.depth = d_model // self.num_heads

        # WQ, WK, WV에 해당하는 밀집층 정의
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        # WO에 해당하는 밀집층 정의
        self.dense = tf.keras.layers.Dense(units=d_model)

    # num_heads 개수만큼 q, k, v를 split하는 함수
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(
            inputs, perm=[0, 2, 1, 3]
        )  # (batch, heads, max 문장 토큰 갯수, 64)

    def call(self, inputs):
        query, key, value, mask = (
            inputs["query"],
            inputs["key"],
            inputs["value"],
            inputs["mask"],
        )
        batch_size = tf.shape(query)[0]

        # 1. WQ, WK, WV에 해당하는 밀집층 지나기
        # q : (batch_size, query의 문장 길이, d_model)
        # k : (batch_size, key의 문장 길이, d_model)
        # v : (batch_size, value의 문장 길이, d_model)
        # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 2. 헤드 나누기
        # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
        # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
        # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
        # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
        # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 4. 헤드 연결(concatenate)하기
        # (batch_size, query의 문장 길이, d_model)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        # 5. WO에 해당하는 밀집층 지나기
        # (batch_size, query의 문장 길이, d_model)
        outputs = self.dense(concat_attention)

        return outputs

In [15]:
def create_padding_mask(x):
    # x (batch_size, max 문장 토큰 수)
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]


# 가릴곳: 1, 참조할곳: 0
def create_look_ahead_mask(x):

    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [27]:
emb = tf.keras.layers.Embedding(input_dim=20, output_dim=2, name='test')
pos = tf.range(start=0, limit=20, delta=1)
emb(pos)

<tf.Tensor: shape=(20, 2), dtype=float32, numpy=
array([[-0.04477648,  0.02605108],
       [ 0.02280302,  0.02852701],
       [-0.0255352 ,  0.04006446],
       [ 0.03522268, -0.01556621],
       [ 0.01954217,  0.02878943],
       [ 0.04505612,  0.02069017],
       [ 0.04850221,  0.02092388],
       [ 0.03063026,  0.03347791],
       [ 0.04661665, -0.01304621],
       [ 0.00839097, -0.03884982],
       [ 0.00209443, -0.0461431 ],
       [-0.0206119 , -0.032855  ],
       [-0.04517735,  0.03406909],
       [ 0.03843451, -0.03344806],
       [-0.04748671,  0.02191739],
       [-0.04776806, -0.04471452],
       [ 0.04883963, -0.02429321],
       [-0.04370322,  0.00925354],
       [ 0.03118126,  0.00491212],
       [ 0.01271736, -0.04705909]], dtype=float32)>

In [28]:
# https://github.com/keras-team/keras-nlp/blob/4aa0503073e21c86333dfc551e6d3443e1bdd017/keras_nlp/src/layers/modeling/position_embedding.py#L21
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(
        self,
        max_sequence_length,
        d_model,
        initializer,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.embedding = tf.keras.layers.Embedding(
            input_dim=max_sequence_length,
            output_dim=d_model,
            name=self.name,
            embeddings_initializer=initializer
        )

    def call(self, inputs):
        seq_length = tf.shape(inputs)[-2] #(...., seq_length, d_model)
        pos = tf.range(start=0, limit=seq_length, delta=1)
        return inputs + self.embedding(pos)

    
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(
        inputs={
            "query": inputs,
            "key": inputs,
            "value": inputs,
            "mask": look_ahead_mask,
        }
    )
    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)


    # 두 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation="relu")(attention1)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention1)
    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask],
        outputs=outputs,
        name=name,
    )


def decoder(vocab_size, num_layers, units, d_model, num_heads, dropout, seq_length, name="decoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 임베딩
    embeddings = PositionalEmbedding(
        initializer = tf.keras.initializers.RandomNormal(
            mean=0.0, 
            stddev=0.02
        ),        
        name='positional_embedding',
        max_sequence_length=seq_length,
        d_model=d_model,
    )(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="decoder_layer_{}".format(i),
        )(inputs=[outputs, look_ahead_mask])

    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask],
        outputs=outputs,
        name=name,
    )


In [29]:
def GPT1(
    vocab_size, num_layers, units, d_model, num_heads, dropout, seq_length, name="transformer",
):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask, output_shape=(1, None, None), name="look_ahead_mask"
    )(inputs)

    # 디코더
    outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        seq_length=seq_length,
    )(inputs=[inputs, look_ahead_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(outputs)

    return tf.keras.Model(
        inputs=inputs,
        outputs=outputs,
        name=name,
    )

[GPT1 Paper](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf)

#### Model specifications 
Our model largely follows the original transformer work [62]. We trained a
12-layer decoder-only transformer with masked self-attention heads (768 dimensional states and 12
attention heads). For the position-wise feed-forward networks, we used 3072 dimensional inner states.
We used the Adam optimization scheme [27] with a max learning rate of 2.5e-4. The learning rate
was increased linearly from zero over the first 2000 updates and annealed to 0 using a cosine schedule.
We train for 100 epochs on minibatches of 64 randomly sampled, contiguous sequences of 512 tokens.
Since layernorm [2] is used extensively throughout the model, a simple weight initialization of
N(0, 0.02) was sufficient. We used a bytepair encoding (BPE) vocabulary with 40,000 merges [53]
and residual, embedding, and attention dropouts with a rate of 0.1 for regularization. We also
employed a modified version of L2 regularization proposed in [37], with w = 0.01 on all non bias or
gain weights. For the activation function, we used the Gaussian Error Linear Unit (GELU) [18]. We
used learned position embeddings instead of the sinusoidal version proposed in the original work.
We use the ftfy library2
to clean the raw text in BooksCorpus, standardize some punctuation and
whitespace, and use the spaCy tokenizer.3

### Fine-tuning details
Unless specified, we reuse the hyperparameter settings from unsupervised
pre-training. We add dropout to the classifier with a rate of 0.1. For most tasks, we use a learning rate
of 6.25e-5 and a batchsize of 32. Our model finetunes quickly and 3 epochs of training was sufficient
for most cases. We use a linear learning rate decay schedule with warmup over 0.2% of training. λ
was set to 0.5.

In [32]:
tf.keras.backend.clear_session()

def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction="none"
    )(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)


# 하이퍼파라미터
# https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf
# Model speci
NUM_LAYERS = 12  # GPT의 경우 인코더 층의 개수로 설정됨
D_MODEL = 768  # GPT 모델에서 사용된 모델 크기
NUM_HEADS = 12  # GPT에서 사용된 멀티 헤드 어텐션의 헤드 수
UNITS = 3072  # GPT에서 사용된 피드 포워드 신경망의 은닉층 크기
DROPOUT = 0.1  # GPT에서 사용된 드롭아웃 비율
EPOCHS=100

model = GPT1(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    seq_length=MAX_LENGTH,
)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
look_ahead_mask (Lambda)        (None, 1, None, None 0           inputs[0][0]                     
__________________________________________________________________________________________________
decoder (Functional)            (None, None, 768)    91842816    inputs[0][0]                     
                                                                 look_ahead_mask[0][0]            
__________________________________________________________________________________________________
outputs (Dense)                 (None, None, 8819)   6781811     decoder[0][0]          

In [33]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
)
def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(
    optimizer=optimizer, 
    loss=loss_function, 
    metrics=[accuracy],
)

history = model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/100
299/299 [==============================] - 90s 259ms/step - loss: 2.6682 - accuracy: 0.0941
Epoch 2/100
299/299 [==============================] - 76s 254ms/step - loss: 2.3816 - accuracy: 0.1052
Epoch 3/100
299/299 [==============================] - 75s 252ms/step - loss: 2.4929 - accuracy: 0.0733
Epoch 4/100
299/299 [==============================] - 75s 252ms/step - loss: 2.5585 - accuracy: 0.0567
Epoch 5/100
299/299 [==============================] - 75s 251ms/step - loss: 2.4917 - accuracy: 0.0728
Epoch 6/100
299/299 [==============================] - 75s 250ms/step - loss: 2.3615 - accuracy: 0.1025
Epoch 7/100
299/299 [==============================] - 75s 250ms/step - loss: 2.3155 - accuracy: 0.1046
Epoch 8/100
299/299 [==============================] - 75s 250ms/step - loss: 2.2938 - accuracy: 0.1061
Epoch 9/100
299/299 [==============================] - 75s 250ms/step - loss: 2.2884 - accuracy: 0.1061
Epoch 10/100
299/299 [==============================] - 75s 250m

In [104]:
def top_k_sample(predictions, top_k):
    top_k_indices = tf.argsort(predictions, axis=-1, direction='DESCENDING')[:, :, :top_k]
    random_index = tf.random.uniform(shape=[], minval=0, maxval=top_k, dtype=tf.int32)
    predicted_id = tf.gather(top_k_indices, random_index, axis=-1) # predicted_id = top_k_indices[0, 0, random_index]
    predicted_id = tf.cast(predicted_id, tf.int32)
    return predicted_id

def decoder_inference(sentence, top_k=10):
    sentence = preprocess(sentence)
    inputs = tf.expand_dims(SOS + tokenizer.encode(sentence) + EOS, axis=0)
    outputs = tf.expand_dims(SOS, 0)

    for i in range(MAX_LENGTH):
        predictions = model(inputs[:,i:], training=False)
        predictions = predictions[:, -1:, :]
        
        predicted_id = top_k_sample(predictions, top_k)
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if tf.equal(predicted_id, EOS[0]):  # end_token_id
            break

        inputs = tf.concat([inputs, predicted_id], axis=-1)
        outputs = tf.concat([outputs, predicted_id], axis=-1)
    return tf.squeeze(outputs, axis=0)


def sentence_generation(sentence):
    prediction = decoder_inference(sentence)
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size]
    )
    print(f"🧑 : {sentence}")
    print(f"🤖 : {predicted_sentence}")

In [105]:
for _, q in df.sample(5).iterrows():
    question = q['text']
    sentence_generation(question)

🧑 : 무슨마음일까
🤖 : 썸 내가 너무 좋은 좋은 너무 남자친구가 잘 오늘 좋은 좋은 좋은 남자친구가 나 잘 오늘 너무 좋아하는 남자친구가 나 
🧑 : 손길이 가서 더 애정이 생길 것 같아요 .
🤖 : 오늘 오늘 나 너무 남자친구가 남자친구가 내가 잘 이제 남자친구가 오늘 나 너무 잘 이제 나 좋아하는 썸 이제 잘 
🧑 : 배고파
🤖 : 나 내가 내가 좋은 이제 이제 내가 내가 내가 너무 오늘 좋은 좋은 남자친구가 내가 좋은 나 내가 너무 나 
🧑 : 썸녀가 그냥 오빠 동생으로 지내자 했음 .
🤖 : 좋아하는 이제 너무 오늘 내가 오늘 이제 오늘 좋은 남자친구가 너무 좋은 나 내가 너무 너무 좋은 좋은 남자친구가 나 
🧑 : 힘든 것 좀 끝났으면
🤖 : 이제 나 내가 좋아하는 남자친구가 남자친구가 썸 좋아하는 잘 남자친구가 오늘 남자친구가 좋은 좋아하는 좋아하는 너무 오늘 오늘 잘 이제 


In [107]:
sentence_generation("느그서장 남천동 살제?")

🧑 : 느그서장 남천동 살제?
🤖 : 이제 좋아하는 내가 너무 썸 나 남자친구가 남자친구가 썸 좋은 너무 오늘 오늘 잘 이제 나 오늘 나 좋은 이제 


## retrospect

```py
predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
```

- 위 방식을 사용하니 '좋아하는'이라는 토큰만 나오는 문제점이 발생했다. 이 때문에 top 10개를 선별해서, random하게 return해주었다.


>Q. inputs에 prediction을 추가해서 for loop를 돌면서 예측해야 할지, 아니면 inputs[:,i]를 통해서, 문장이 predict될 때마다 앞쪽 token을 지워줘야 하는지 모르겠다.
